In [51]:
from pydantic import BaseModel, Field
from typing import Optional, Dict, List, ForwardRef
from intervaltree import IntervalTree, Interval
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import hashlib

In [12]:
HDR = ['chr', 'src', 'feature_type', 'start', 'end', 'score', 'strand', 'frame', 'attributes']

In [50]:
def load_attributes(s: str, kv_sep: str = '=') -> dict:
    return {
        k.strip(): v.strip()
        for x in s.strip().split(';') if x
        for k, v in [x.strip().split(kv_sep, 1)]
    }

In [ ]:
GFeatureRef = ForwardRef("GFeature")

g2t_map = dict()
g2t_map_2 = dict()

class GFeature(BaseModel):
    chr : str
    src : str
    feature_type : str
    start : int
    end : int
    score : Optional[float]
    strand : str # ['.', '-', '+']
    frame : str # ['.', '0', '1', '2']
    attributes : Dict[str, str]
    _parent : Optional[str] = None
    _id : Optional[str] = None
    _hash : Optional[str] = None

    def __init__(self, **data):
        super().__init__(**data)
        self._infer_id()
        self._hash_id()
        self._infer_parent()

    def _infer_id(self):
        for k, v in self.attributes.items():
            if 'id' in k.lower():
                self._id = v
                break
    
    def _infer_parent(self):
        for k, v in self.attributes.items():
            if 'parent' in k.lower():
                self._parent = v
                break
                
    def _hash_id(self):
        raw = f"{self.chr}{self.strand}-{self.start}-{self.end}"
        self._hash = hashlib.sha256(raw.encode()).hexdigest()
        
    def __repr__(self) -> str:
        return f"{self.feature_type}:{self._id},{self.strand},{self.start}-{self.end},{self._hash}"
    
    def __eq__(self, o) -> bool:
        return o.chr == self.chr and \
            o.strand == self.strand and \
            o.start == self.start and \
            o.end == self.end

class Gene(GFeature):
    children : Dict[str, GFeature]

class Transcript(GFeature):
    children : IntervalTree

class Exon(GFeature):
    child : GFeature
    
def pd_row2gfeature(row):
    try:
        gfeat = GFeature(
            chr = row['chr'],
            src = row['src'],
            feature_type = row['feature_type'],
            start = row['start'],
            end = row['end'],
            score = None if row['score'] == '.' else row['score'],
            strand = row['strand'],
            frame = row['frame'],
            attributes = load_attributes(row['attributes'])
        )
    except Exception as e:
        raise RuntimeError(f"error while converting pd series to GFeature: {e}")
    return gfeat

SyntaxError: invalid syntax (3893811245.py, line 53)

In [148]:
fn = "/ccb/salz4-3/hji20/hg002-q100-annotation/data/chm13v2.0_RefSeq_Liftoff_v5.2.gff3"

In [149]:
df = pd.read_csv(fn, sep='\t', comment='#', header=None)
df.columns = HDR
df.head()

,chr,src,feature_type,start,end,score,strand,frame,attributes
0,chr1,Liftoff,gene,6047,13941,.,-,.,ID=LOC124900618;gene_name=LOC124900618;db_xref...
1,chr1,Liftoff,transcript,6047,13941,.,-,.,ID=XR_002958507.2;Parent=LOC124900618;db_xref=...
2,chr1,Liftoff,exon,6047,6420,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...
3,chr1,Liftoff,exon,12078,12982,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...
4,chr1,Liftoff,exon,13445,13579,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...


In [17]:
df.head()

,chr,src,feature_type,start,end,score,strand,frame,attributes
0,chr1,Liftoff,gene,6047,13941,.,-,.,ID=LOC124900618;gene_name=LOC124900618;db_xref...
1,chr1,Liftoff,transcript,6047,13941,.,-,.,ID=XR_002958507.2;Parent=LOC124900618;db_xref=...
2,chr1,Liftoff,exon,6047,6420,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...
3,chr1,Liftoff,exon,12078,12982,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...
4,chr1,Liftoff,exon,13445,13579,.,-,.,Parent=XR_002958507.2;db_xref=GeneID:124900618...


In [107]:
feature_types = df['feature_type'].unique()
df_by_feature_type = dict()

for x in feature_types:
    df_by_feature_type[x] = df[df['feature_type'] == x]

In [79]:
def objectify_df(df):
    res = df.apply(pd_row2gfeature, axis=1)
    return res

In [ ]:
genes = dict() # root nodes
# separate genes dict() per chr
for _, row in df_by_feature_type['gene'].iterrows():
    gfeat = pd_row2gfeature(row)
    genes[gfeat._id] = gfeat

In [150]:
dfs_by_chr = [x for _, x in df_by_feature_type['gene'].groupby("chr")]

In [154]:
for x in df_by_feature_type['gene'].groupby("chr"):
    print(x)

('chr1',          chr      src feature_type      start        end score strand frame  \
0       chr1  Liftoff         gene       6047      13941     .      -     .   
6       chr1  Liftoff         gene      15080      21429     .      +     .   
10      chr1  Liftoff         gene      20529      37628     .      -     .   
22      chr1  Liftoff         gene      52979      54612     .      -     .   
27      chr1  Liftoff         gene     100966     101472     .      +     .   
...      ...      ...          ...        ...        ...   ...    ...   ...   
392483  chr1  Liftoff         gene  248311348  248368818     .      +     .   
392490  chr1  Liftoff         gene  248312352  248312457     .      +     .   
392494  chr1  Liftoff         gene  248312745  248312816     .      +     .   
392535  chr1  Liftoff         gene  248351319  248351420     .      -     .   
392542  chr1  Liftoff         gene  248375192  248375720     .      +     .   

                                          

In [140]:
# TODO: implement parallel processing
orphans = []
transcripts = dict()
for _, row in df_by_feature_type['transcript'].iterrows():
    gfeat = pd_row2gfeature(row)
    if not gfeat._parent:
        orphans.append(gfeat)
    genes[gfeat._parent].children[gfeat._id] = gfeat
    transcripts[gfeat._id] = gfeat

In [145]:
# TODO: parallel
for _, row in df_by_feature_type['exon'].iterrows():
    gfeat = pd_row2gfeature(row)
    transcripts[gfeat._parent].children[gfeat._hash] = gfeat

In [146]:
transcripts[gfeat._parent].children

{'8d031be0790e5d26836f7c895af67e09f59f74358087992d4b0da56f168a8994': exon:None,-,62449384-62450563,8d031be0790e5d26836f7c895af67e09f59f74358087992d4b0da56f168a8994,
 '61a43713bdd2cc180fde52d861ef624bf6bf4e1395579d1e06b2bd089d1cfd24': exon:None,-,62451063-62451171,61a43713bdd2cc180fde52d861ef624bf6bf4e1395579d1e06b2bd089d1cfd24,
 'fa0a84f01da7349aa022bd9ea8588fbb0c272116a093f56e06c1e5b2a5ececd1': exon:None,-,62451557-62451910,fa0a84f01da7349aa022bd9ea8588fbb0c272116a093f56e06c1e5b2a5ececd1}

In [80]:
with ProcessPoolExecutor(max_workers=24) as executor:
    results = list(executor.map(objectify_df, df_by_feature_type.values()))

Process ForkProcess-100:
Process ForkProcess-108:
Exception ignored in: <function _afterFork at 0x7f281a0af600>
Traceback (most recent call last):
  File "/home/hji20/miniconda3/envs/mjol/lib/python3.13/logging/__init__.py", line 245, in _afterFork
    def _afterFork():
KeyboardInterrupt: 
Process ForkProcess-98:
Process ForkProcess-103:
Process ForkProcess-99:
Process ForkProcess-107:
Process ForkProcess-97:
Process ForkProcess-101:
Process ForkProcess-106:
Process ForkProcess-104:
Process ForkProcess-102:
Process ForkProcess-105:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/hji20/miniconda3/envs/mjol/l

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [72]:
df_grouped_by_chr = [group for _, group in df.groupby("chr")]

In [73]:
def objectify_df(df):
    res = df.apply(pd_row2gfeature, axis=1)
    return res

In [74]:
with ProcessPoolExecutor(max_workers=24) as executor:
    results = list(executor.map(objectify_df, df_grouped_by_chr))

In [68]:
test_chr = results[0]

In [18]:
tree = IntervalTree()
for feature in [gene] + transcripts:
    tree[feature.start:feature.end] = feature

In [19]:
query_start, query_end = 1100, 1400
overlapping = tree[query_start:query_end]
print(f"Features overlapping {query_start}-{query_end}:")
for iv in overlapping:
    print(iv.data)

Features overlapping 1100-1400:
seqid='chr1' src='ensembl' feature_type='gene' start=1000 end=9000 score=None strand='+' frame='.' attributes={'ID': 'gene1'}
seqid='chr1' src='ensembl' feature_type='transcript' start=1200 end=5200 score=None strand='+' frame='.' attributes={'ID': 'tx2', 'Parent': 'gene1'}
seqid='chr1' src='ensembl' feature_type='transcript' start=1100 end=5100 score=None strand='+' frame='.' attributes={'ID': 'tx1', 'Parent': 'gene1'}
seqid='chr1' src='ensembl' feature_type='transcript' start=1300 end=5300 score=None strand='+' frame='.' attributes={'ID': 'tx3', 'Parent': 'gene1'}
